In [44]:
import pandas as pd
import math
from ast import literal_eval

In [19]:
table = pd.read_csv("./csv.csv")
table_n = table.shape[0]

In [109]:
email_dict = {}
new_table1 = []
new_table2 = []
for i in range (table_n) :
    current = {}
    current["SCAMID"] = i
    current["SCAMNUMBER"] = i
    current["DESC"] = table["Description"][i]
    new_table1.append(current)
    if isinstance(table["EMAIL"][i],str) or not(math.isnan(table["EMAIL"][i]) ):
        test_list = (literal_eval(table["EMAIL"][i]))
        res = []
        [res.append(x) for x in test_list if x not in res]
        for x in res:
            current1 = {}
            current1["SCAMID"] = i
            current1["EMAIL"] = x
            new_table2.append(current1)
            if x in email_dict:
                email_dict[x] +=1
            else :
                email_dict[x] = 1

In [110]:
new_table1

[{'SCAMID': 0,
  'SCAMNUMBER': 0,
  'DESC': "I received an email from this company asking if I was interested in working for them. They accepted me.  It was a car wrap business.  They sent check for $1997.00.  I deposited it.  They were give back to them earnest money, $1597.00 in case the wrap got damaged.  They asked me to do it through paypay.  I asked if I could send a check but I ended up doing it through CashApp.   They scheduled me for the vehicle wrap and they were a no show.  When I contacted them, there was no answer.  The bank said I couldn't get the money back."},
 {'SCAMID': 1,
  'SCAMNUMBER': 1,
  'DESC': 'This was after I applied to a certified position on LinkedIn with BBB. I was just contacted by a claimed "recruiter" and told to download wire secure messanger app. I have been scammed with the same email before for a different company'},
 {'SCAMID': 2,
  'SCAMNUMBER': 2,
  'DESC': 'Saw ad on Facebook about making money doing customer surveys, I enquired and was told th

In [112]:
new_table_frame = pd.DataFrame(new_table1) 
new_table_frame.to_csv("scam1.csv",index=False)

In [69]:
{k: v for k, v in sorted(email_dict.items(), key=lambda item: item[1])}

{'jh.odysseycapitalgroup057@gmail.com': 1,
 'gloriacarter@empowerrecuiter.com': 1,
 'sharonshaw026@gmail.com': 1,
 'officedesk043@gmail.com': 1,
 'dillon@on-siteshopper.com': 1,
 'cynthia@laminioncare.com': 1,
 'decofficesupply@gmail.com': 1,
 'Nina.Kaufman@spstrademail.com': 1,
 'Anna.Sotello@spstrademail.com': 1,
 'santmyeroilcompanyinc@gmail.com': 1,
 'dw.santmyercompaniesinc@gmail.com': 1,
 'johnmiley3800@gmail.com': 1,
 'human.resources@sinowayemail.com': 1,
 'lydie.lucas@sinowayemail.com': 1,
 'Christine.Nowell@sinowayemail.com': 1,
 'paule.ruggiero@sinowayemail.com': 1,
 'Alma.Dickson@sinowayemail.com': 1,
 'accounting@sinowayemail.com': 1,
 'elisabetta.mathieu@sinowayemail.com': 1,
 'support@danielbeetrootr.com': 1,
 'milton@squintshopper.com': 1,
 'jasmin.pagani@mindrey-email.com': 1,
 'accounting@mindrey-email.com': 1,
 'a.howe@ams-med.us': 1,
 'alicia@magunalli.com': 1,
 'chartercommunicationsinc@aol.com': 1,
 'Billy_Hargreaves@outlook.com': 1,
 'reyescristina217@gmail.com':

In [70]:
email_dict = {}
new_table1 = []
new_table2 = []
for i in range (table_n) :
    current = {}
    current["SCAMID"] = i
    current["DESC"] = table["Description"][i]
    if isinstance(table["URL"][i],str) or not(math.isnan(table["URL"][i]) ):
        test_list = (literal_eval(table["URL"][i]))
        res = []
        [res.append(x) for x in test_list if x not in res]
        for x in res:
            current1 = {}
            current1["SCAMID"] = i
            current1["URL"] = x
            new_table2.append(current1)
            if x in email_dict:
                email_dict[x] +=1
            else :
                email_dict[x] = 1

In [72]:
{k: v for k, v in sorted(email_dict.items(), key=lambda item: -item[1])}

{'www.google.com/hangouts': 36,
 'Indeed.com': 30,
 'www.gmail.com': 27,
 'indeed.com': 21,
 'BBB.org': 10,
 'https://hangouts.google.com/': 10,
 'jobs.org': 9,
 'gmail.com': 8,
 'webstorejobs.org': 8,
 'https://hangouts.google.com': 5,
 '@gmail.com': 5,
 'ziprecruiter.com': 5,
 'care.com': 5,
 'www.usps.com': 5,
 'stockretail.org': 5,
 'mail.google.com': 4,
 'www.google.com/hangouts/': 4,
 'aol.com': 4,
 'bbb.org': 4,
 'https://docs.google.com/document/d/1ecGeM-NfJOAVv5Zm9xezteMdiPlei7VN8hN9tRawt88/': 4,
 'https://www.logistics-station.com/': 4,
 'http://www.gmail.com': 4,
 'careerbuilder.com': 4,
 'ZipRecruiter.com': 4,
 'www.adobe.com': 4,
 'monster.com': 4,
 'www.fedex.com': 3,
 '************@gmail.com': 3,
 'smtpsendemail.com': 3,
 'http://alpecourierservices.com/career': 3,
 'https://orders-crm.net': 3,
 'www.google.com/hangout': 3,
 'freelancer.com': 3,
 'http://www.google.com/hangouts/': 3,
 'retailpay.org': 3,
 'shopperweb.org': 3,
 'storejobs.org': 3,
 'http://www.pinnemploym

In [73]:
email_dict = {}
new_table1 = []
new_table2 = []
for i in range (table_n) :
    current = {}
    current["SCAMID"] = i
    current["DESC"] = table["Description"][i]
    if isinstance(table["ORGANIZATION"][i],str) or not(math.isnan(table["ORGANIZATION"][i]) ):
        test_list = (literal_eval(table["ORGANIZATION"][i]))
        res = []
        [res.append(x) for x in test_list if x not in res]
        for x in res:
            current1 = {}
            current1["SCAMID"] = i
            current1["ORGANIZATION"] = x
            new_table2.append(current1)
            if x in email_dict:
                email_dict[x] +=1
            else :
                email_dict[x] = 1

In [74]:
{k: v for k, v in sorted(email_dict.items(), key=lambda item: -item[1])}

{'Google': 682,
 'Walmart': 380,
 'Amazon': 183,
 'Google Hangouts': 156,
 'Skype': 141,
 'Facebook': 132,
 'UPS': 131,
 'Apple': 120,
 'Customer Service': 120,
 'Microsoft': 119,
 'BBB': 111,
 'USPS': 109,
 'Bank': 99,
 'Company': 82,
 'Employee Wellness': 78,
 'FBI': 78,
 'Better Business Bureau': 77,
 'Fedex': 73,
 'AD&D Insurance': 71,
 'LLC': 69,
 'Western Union': 62,
 'Bank of America': 58,
 'CVS': 57,
 'Walgreens': 57,
 'Wal-Mart': 54,
 'Wells Fargo': 52,
 'PayPal': 51,
 'Kroger': 50,
 'FTC': 47,
 'Microsoft Office': 43,
 'Fed': 36,
 'Paypal': 31,
 'IRS': 29,
 'Ebay': 28,
 'Verizon': 28,
 'MoneyGram': 28,
 'HR Department': 25,
 'Moneygram': 24,
 'Fax Machine': 23,
 'AD & D Insurance': 22,
 'Linkedin': 21,
 'Peachtree Complete Accounting Software': 20,
 'USAA': 20,
 'eBay': 20,
 'Apple Store': 20,
 'Health Insurance': 19,
 'Macbook': 18,
 'Walgreen': 17,
 'DHL': 17,
 'Human Resources Department': 17,
 'Federal Trade Commission': 17,
 'Chase Bank': 17,
 'Kmart': 17,
 'Life Insuran

In [93]:
new_table2 = []
for i in range (table_n) :
    current1 = {}
    current1["SCAMID"] = i
    x = table["Company"][i]
    if isinstance(x,str):
        current1["SCAMMER"] = x.lower() 
    else: 
        current1["SCAMMER"] = '' 
    new_table2.append(current1)


In [101]:
new_table_frame = pd.DataFrame(new_table2) 
new_table_frame.to_csv("scammer.csv",index=False)

In [86]:
email_dict = {}
new_table1 = []
new_table2 = []
for i in range (table_n) :
    current = {}
    current["SCAMID"] = i
    current["DESC"] = table["Description"][i]
    if isinstance(table["COUNTRY"][i],str) or not(math.isnan(table["COUNTRY"][i]) ):
        test_list = (literal_eval(table["COUNTRY"][i]))
        res = []
        [res.append(x.lower()) for x in test_list if x.lower() not in res]
        for x in res:
            current1 = {}
            current1["SCAMID"] = i
            current1["COUNTRY"] = x
            new_table2.append(current1)
            if x in email_dict:
                email_dict[x] +=1
            else :
                email_dict[x] = 1

In [87]:
{k: v for k, v in sorted(email_dict.items(), key=lambda item: -item[1])}

{'us': 252,
 'usa': 150,
 'canada': 133,
 'united states': 124,
 'u.s.': 69,
 'america': 49,
 'uk': 37,
 'china': 35,
 'germany': 34,
 'the states': 34,
 'australia': 31,
 'india': 27,
 'united states of america': 23,
 'japan': 20,
 'united kingdom': 19,
 'italy': 15,
 'switzerland': 15,
 'ukraine': 14,
 'u.s.a.': 13,
 'france': 12,
 'sweden': 10,
 'u.s.a': 10,
 'england': 10,
 'singapore': 10,
 'russia': 9,
 'philippines': 8,
 'mexico': 7,
 'spain': 7,
 'poland': 7,
 'ph': 6,
 'denmark': 6,
 'turkey': 6,
 'austria': 5,
 'brazil': 4,
 'new zealand': 4,
 'south africa': 4,
 'panama': 4,
 'portugal': 3,
 'indonesia': 3,
 'norway': 3,
 'netherlands': 3,
 'nicaragua': 3,
 'ch': 3,
 'nigeria': 3,
 'jamaica': 3,
 'ireland': 3,
 'belgium': 3,
 'scotland united kingdom': 3,
 'colombia': 2,
 'namibia': 2,
 'malawi': 2,
 'iraq': 2,
 'uae': 2,
 'czech republic': 2,
 'cyprus': 2,
 'romania': 2,
 'venezuela': 2,
 'united arab emirates': 2,
 'kuwait': 2,
 'iceland': 2,
 'cuba': 1,
 'south korea': 1,

In [ ]:
email_dict = {}
new_table1 = []
new_table2 = []
for i in range (table_n) :
    current = {}
    current["SCAMID"] = i
    current["DESC"] = table["Description"][i]
    if isinstance(table["COUNTRY"][i],str) or not(math.isnan(table["COUNTRY"][i]) ):
        test_list = (literal_eval(table["COUNTRY"][i]))
        res = []
        [res.append(x.lower()) for x in test_list if x.lower() not in res]
        for x in res:
            current1 = {}
            current1["SCAMID"] = i
            current1["COUNTRY"] = x
            new_table2.append(current1)
            if x in email_dict:
                email_dict[x] +=1
            else :
                email_dict[x] = 1